### 2. Знакомство с новыми данными: данные о квартирах от Сбера

In [ ]:
# В этом модуле мы с вами будем работать с данными с самого настоящего соревнования на платформе Kaggle, 
# инициатором которого стал Сбер. Соревнование проводилось в 2017 году, его призовой фонд составил 25 000 $. 
# Требования Сбера состояли в построении модели, которая бы прогнозировала цены на жильё в Москве, 
# опираясь на параметры самого жилья, а также состояние экономики и финансового сектора в стране.

# Датасет представляет собой набор данных из таблицы с информацией о параметрах жилья (train.csv). 
# В ней содержатся 292 признака о состоянии экономики России на момент продажи недвижимости (macro.csv). 

# Для упрощения техники очистки данных мы будем отрабатывать на урезанном датасете.

# Скачать набор данных в формате csv (разделитель — ',') 
# можно здесь (csv-файл находится в zip-архиве — распакуйте архив, прежде чем продолжать работу!)

# Он содержит информацию о 61 признаке. Их значение мы будем объяснять в процессе работы с данными.

# Импортируем библиотеки, которые нам понадобятся (pandas для работы с данными, 
# numpy для математических преобразований, matplotlib и seaborn для визуализации):

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Прочитаем наши данные и выведем первые пять строк таблицы с помощью head(), чтобы убедиться в том, что всё подгрузилось верно:
sber_data = pd.read_csv('data/sber_data.csv')
sber_data.head()

### 3. Работа с пропусками: как их обнаружить?

In [ ]:
# Добавить страницу в мои закладки
# Работа с отсутствующими записями в таблице, пожалуй, одна из самых сложных и неоднозначных. 
# Она же — самая распространённая для реальных данных. С неё мы и начнём!

# В pandas пропуски обозначаются специальным символом NaN (Not-a-Number — «не число»). 

# Мы с вами уже сталкивались с пропусками, как только начали работать с настоящими данными. 
# Вспомните датасеты о домах в Мельбурне и коронавирусе. 

# Чем так плохи пропуски, почему так важно их предобработать?

# Ответ очень прост: преобладающее большинство моделей машинного обучения не умеют обрабатывать пропуски, 
# так как они работают только с числами. Если в данных содержится пустая ячейка таблицы, модель выдаст ошибку. 

# ПРИЧИНЫ ПОЯВЛЕНИЯ ПРОПУСКОВ В ДАННЫХ

# Ошибка ввода данных. Как правило, такая ошибка обусловлена человеческим фактором: никто не застрахован 
# от случайного пропуска графы при заполнении данных.
# Ошибка передачи данных. Эта причина на сегодняшний момент возникает довольно редко: 
# с появлением протоколов проверки выгружаемой информации потерять данные при передаче их по сети становится сложнее, 
# но вероятность такого события ненулевая.
# Намеренное сокрытие информации. Одна из самых распространённых причин, особенно в социологических опросах.
# Дело в том, что пользователи/опрашиваемые/клиенты часто скрывают информацию о себе. 
# Например, люди, занимающие высокие должности, могут быть связаны контрактом о неразглашении своих доходов. 
# Прямое отсутствие информации. Эта причина очень распространена в данных для рекомендательных систем. 
# Представьте, что у нас есть таблицы фильмов и пользователей, которые просматривают их и ставят им оценки. 
# Мы объединяем всю информацию в одну большую сводную таблицу: например, по строкам идут пользователи, 
# а по столбцам — фильмы. Но вот незадача: у нас нет информации о рейтингах фильмов, которые пользователь ещё не посмотрел. 
# В таком случае на пересечении строки с именем пользователя и столбца с названием фильма, который он ещё не смотрел, ставится пропуск. 
# Главное несчастье состоит в том, что 99 % процентов такой таблицы — это сплошной пропуск.
# Мошенничество. Очень острая проблема в финансовой сфере, особенно в банковских данных. 
# Мошенники нередко указывают ложную информацию или не указывают её вовсе.

# КАК ОБНАРУЖИТЬ ПРОПУСКИ?
# Ранее мы определяли наличие пропусков в данных с помощью метода info(). 
# Но этот метод не позволяет точно локализовать места пропущенных значений, 
# он выводит только число непустых значений и предназначен для определения факта наличия пропусков:

# Найти пропуски зачастую довольно просто за исключением тех случаев, когда пропуски скрыты.

# В библиотеке pandas специально для этого реализован метод isnull(). Этот метод возвращает новый DataFrame, 
# в ячейках которого стоят булевы значения True и False. True ставится на месте, где ранее находилось значение NaN.
# Посмотрим на результат работы метода на нашей таблице:
print(sber_data.isnull().tail())

# Из таблицы можно увидеть, где были пропущены значения: ячейки со значением True; ячейки, где стоит False, были изначально заполнены.

# Как вы сами понимаете, результат метода isnull() — это, мягко говоря, не самый удобный метод поиска пропусков, 
# однако он является промежуточным этапом других способов, которые мы рассмотрим далее.

In [ ]:
# СПИСОК СТОЛБЦОВ С ПРОПУСКАМИ

# Первый способ — это вывести на экран названия столбцов, где число пропусков больше 0. 
# Для этого вычислим средний по столбцам результат метода isnull(). Получим долю пропусков в каждом столбце.
# Умножаем на 100 %, находим столбцы, где доля пропусков больше 0, сортируем по убыванию и выводим результат:

cols_null_percent = sber_data.isnull().mean() * 100
cols_with_null = cols_null_percent[cols_null_percent>0].sort_values(ascending=False)
print(cols_with_null)

# Итак, можно увидеть, что у нас большое число пропусков (более 47 %) в столбце hospital_beds_raion 
# (количество больничных коек в округе). 

# Далее у нас идут столбцы с числом пропусков чуть больше 20 %: 

# preschool_quota (число мест в детском саду в районе);
# school_quota (число мест в школах в районе);
# life_sq (жилая площадь здания в квадратных метрах). 
# Менее одного процента пропусков содержат признаки:

# floor (число этажей в доме);
# metro_min_walk (время от дома до ближайшего метро пешком в минутах);
# metro_km_walk (расстояние до ближайшего метро в километрах);
# railroad_station_walk_km (расстояние до ближайшей ж. д. станции в километрах);
# railroad_station_walk_min (время до ближайшей ж. д. станции пешком в минутах). 

In [ ]:
# СТОЛБЧАТАЯ ДИАГРАММА ПРОПУСКОВ

# Иногда столбцов с пропусками становится слишком много и прочитать информацию о них из списка признаков 
# с цифрами становится слишком затруднительно — цифры начинают сливаться воедино. 

# Можно воспользоваться столбчатой диаграммой, чтобы визуально оценить соотношение числа пропусков к числу записей. 
# Самый быстрый способ построить её — использовать метод plot():
cols_with_null.plot(
    kind='bar',
    figsize=(10, 4),
    title='Распределение пропусков в данных'
);

In [ ]:
# ТЕПЛОВАЯ КАРТА ПРОПУСКОВ 

# Ещё один распространённый способ визуализации пропусков — тепловая карта. 

# Её часто используют, когда столбцов с пропусками не так много (меньше 10). 
# Она позволяет понять не только соотношение пропусков в данных, но и их характерное местоположение в таблице. 

# Для создания такой тепловой карты можно воспользоваться результатом метода isnull(). 
# Ячейки таблицы, в которых есть пропуск, будем отмечать жёлтым цветом, а остальные — синим. 
# Для этого создадим собственную палитру цветов тепловой карты с помощью метода color_pallete() из библиотеки seaborn.
colors = ['blue', 'yellow'] 
fig = plt.figure(figsize=(10, 4))
cols = cols_with_null.index
ax = sns.heatmap(
    sber_data[cols].isnull(),
    cmap=sns.color_palette(colors),
)

### 4. Работа с пропусками: методы обработки

In [ ]:
# ОТБРАСЫВАНИЕ ЗАПИСЕЙ И ПРИЗНАКОВ

# Первая техника — самая простая из всех. Она предполагает простое удаление записей или признаков, в которых содержатся пропуски. 

# Здесь важно правильно выбрать ось удаления: если мы избавимся от большого числа строк, 
# то рискуем потерять важные данные, а если мы удалим столбцы, то можем потерять важные признаки.
# Прежде всего порассуждаем логически: в столбце hospital_beds_raion более 47 % пропусков. 
# Если мы будем удалять все строки, в которых этот признак пропущен, мы потеряем почти половину наших данных! 

# Правильнее будет просто удалить столбец, ведь число мест 
# в районных больницах — это не самый информативный признак для определения цены квартиры.

# А вот если мы удалим весь столбец metro_km_walk, где менее 1 % пропусков, 
# то потеряем полезный признак при формировании прогноза цены, 
# ведь расстояние до ближайшего метро — это важный фактор при выборе квартиры. В данном случае лучше будет удалить сами записи.

# Специалисты рекомендуют при использовании метода удаления придерживаться следующих правил: 
# удаляйте столбец, если число пропусков в нем более 30-40 %. В остальных случаях лучше удалять записи.

# Для удаления строк и столбцов будем использовать метод dropna(), который позволяет удалять пропуски с тонким подходом к настройке. 

# Основные параметры метода:

# axis — ось, по которой производится удаление (по умолчанию 0 — строки).
# how — как производится удаление строк (any — если хотя бы в одном из столбцов есть пропуск, стоит по умолчанию; 
# all — если во всех столбцах есть пропуски). 
# thresh — порог удаления. Определяет минимальное число непустых значений в строке/столбце, при котором она/он сохраняется. 
# Например, если мы установим thresh в значение 2, то мы удалим строки, где число пропусков  и более, где  — число признаков (если ).
# Предварительно создадим копию исходной таблицы — drop_data, чтобы не повредить её. 
# Зададимся порогом в 70 %: будем оставлять только те столбцы, в которых 70 и более процентов записей не являются пустыми. 
# После этого удалим записи, в которых содержится хотя бы один пропуск. 
# Наконец, выведем информацию о числе пропусков и наслаждаемся нулями.

#создаем копию исходной таблицы
drop_data = sber_data.copy()
#задаем минимальный порог: вычисляем 70% от числа строк
thresh = drop_data.shape[0]*0.7
#удаляем столбцы, в которых более 30% (100-70) пропусков
drop_data = drop_data.dropna(how='any', thresh=thresh, axis=1)
#удаляем записи, в которых есть хотя бы 1 пропуск
drop_data = drop_data.dropna(how='any', axis=0)
#отображаем результирующую долю пропусков
drop_data.isnull().mean()
# Посмотрим на результирующее число записей:
print(drop_data.shape)

In [ ]:
# ЗАПОЛНЕНИЕ НЕДОСТАЮЩИХ ЗНАЧЕНИЙ КОНСТАНТАМИ

# Чтобы дырявая бочка не протекала, вставьте в дырку пробку. 
# С этой мыслью связан другой способ бороться с пропусками — заполнение константами. 

# Чаще всего пустые места заполняют средним/медианой/модой для числовых признаков и модальным значением для категориальных признаков.

# Вся сложность заключается в выборе метода заполнения. 
# Важным фактором при выборе метода является распределение признаков с пропусками. Давайте выведем их на экран. 

# В pandas это можно сделать с помощью метода hist():
cols = cols_with_null.index
sber_data[cols].hist(figsize=(20, 8));

In [ ]:
# Итак, рассмотрим несколько рекомендаций.

# Для распределений, похожих на логнормальное, где пик близ нуля, а далее наблюдается постепенный спад частоты, 
# высока вероятность наличия выбросов (о них мы поговорим чуть позже). Математически доказывается, 
# что среднее очень чувствительно к выбросам, а вот медиана — нет. 
# Поэтому предпочтительнее использовать медианное значение для таких признаков.
# Если признак числовой и дискретный (например, число этажей, школьная квота), то их заполнение средним/медианой является ошибочным, 
# так как может получиться число, которое не может являться значением этого признака. 
# Например, количество этажей — целочисленный признак, а расчёт среднего может дать 2.871. 
# Поэтому такой признак заполняют либо модой, либо округляют до целого числа 
# (или нужного количества знаков после запятой) среднее/медиану.
# Категориальные признаки заполняются либо модальным значением, либо, если вы хотите оставить информацию о пропуске в данных, 
# значением 'unknown'. На наше счастье, пропусков в категориях у нас нет.
# Иногда в данных бывает такой признак, основываясь на котором, можно заполнить пропуски в другом. 
# Например, в наших данных есть признак full_sq (общая площадь квартиры). Давайте исходить из предположения, 
# что, если жилая площадь (life_sq) неизвестна, то она будет равна суммарной площади!
# Заполнение значений осуществляется с помощью метода fillna(). 
# Главный параметр метода — value (значение, на которое происходит заполнение данных в столбце).
# Если метод вызывается от имени всего DataFrame, то в качестве value можно использовать словарь, 
# где ключи — названия столбцов таблицы, а значения словаря — заполняющие константы. 

# Создадим такой словарь, соблюдая рекомендации, приведённые выше, а также копию исходной таблицы. 
# Произведём операцию заполнения с помощью метода fillna() и удостоверимся, что пропусков в данных больше нет:

#создаем копию исходной таблицы
fill_data = sber_data.copy()
#создаем словарь имя столбца: число(признак) на который надо заменить пропуски
values = {
    'life_sq': fill_data['full_sq'],
    'metro_min_walk': fill_data['metro_min_walk'].median(),
    'metro_km_walk': fill_data['metro_km_walk'].median(),
    'railroad_station_walk_km': fill_data['railroad_station_walk_km'].median(),
    'railroad_station_walk_min': fill_data['railroad_station_walk_min'].median(),
    'hospital_beds_raion': fill_data['hospital_beds_raion'].mode()[0],
    'preschool_quota': fill_data['preschool_quota'].mode()[0],
    'school_quota': fill_data['school_quota'].mode()[0],
    'floor': fill_data['floor'].mode()[0]
}

#заполняем пропуски в соответствии с заявленным словарем
fill_data = fill_data.fillna(values)
#выводим результирующую долю пропусков
fill_data.isnull().mean()
# Посмотрим, на то, как изменились распределения наших признаков:
cols = cols_with_null.index
fill_data[cols].hist(figsize=(20, 8));

# Обратите внимание на то, как сильно изменилось распределение для признака hospital_beds_raion. 
# Это связано с тем, что мы заполнили модальным значением почти 47 % общих данных. 
# В результате мы кардинально исказили исходное распределение признака, что может плохо сказаться на модели.

# Недостаток метода заполнения константой состоит в том, что мы можем «нафантазировать» новые данные, 
# которые не учитывают истинного распределения.

In [ ]:
# ЗАПОЛНЕНИЕ НЕДОСТАЮЩИХ ЗНАЧЕНИЙ КОНСТАНТАМИ С ДОБАВЛЕНИЕМ ИНДИКАТОРА

# Если мы используем заполнение пропусков константами, может быть, имеет смысл сказать модели о том, что на этом месте был пропуск? 

# Давайте добавим к нашим данным признаки-индикаторы, которые будут сигнализировать о том, 
# что в столбце на определённом месте в таблице был пропуск. Это место в столбце-индикаторе будем помечать как True. 

# Эта эвристика пытается снизить влияние искажения признака, указав модели на места, где мы «нафантазировали» данные.
# Посмотрим на реализацию. Как обычно, создадим копию indicator_data исходной таблицы. 
# В цикле пройдёмся по столбцам с пропусками и будем добавлять в таблицу новый признак (с припиской "was_null"), 
# который получается из исходного с помощью применения метода isnull(). После чего произведём обычное заполнение пропусков,
# которое мы совершали ранее, и выведем на экран число отсутствующих значений в столбце, чтобы убедиться в результате:

#создаем копию исходной таблицы
indicator_data = sber_data.copy()
#в цикле пробегаемся по названиям столбцов с пропусками
for col in cols_with_null.index:
    #создаем новый признак-индикатор как col_was_null
    indicator_data[col + '_was_null'] = indicator_data[col].isnull()
#создаем словарь имя столбца: число(признак) на который надо заменить пропуски   
values = {
    'life_sq': indicator_data['full_sq'],
    'metro_min_walk': indicator_data['metro_min_walk'].median(),
    'metro_km_walk': indicator_data['metro_km_walk'].median(),
    'railroad_station_walk_km': indicator_data['railroad_station_walk_km'].median(),
    'railroad_station_walk_min': indicator_data['railroad_station_walk_min'].median(),
    'hospital_beds_raion': indicator_data['hospital_beds_raion'].mode()[0],
    'preschool_quota': indicator_data['preschool_quota'].mode()[0],
    'school_quota': indicator_data['school_quota'].mode()[0],
    'floor': indicator_data['floor'].mode()[0]
}
#заполняем пропуски в соответствии с заявленным словарем
indicator_data = indicator_data.fillna(values)
#выводим результирующую долю пропусков
indicator_data.isnull().mean()

# Метод исходит из предположения, что, если дать модели информацию о том, что в ячейке ранее была пустота, 
# то она будет меньше доверять таким записям и меньше учитывать её в процессе обучения. Иногда такие фишки действительно работают, 
# иногда не дают эффекта, а иногда и вовсе могут ухудшить результат обучения и затруднить процесс обучения.

# Когда мы добавляем признаки-индикаторы, мы повышаем размерность наших данных. Теперь представьте, что столбцов с пропусками сотня, 
# по паре штук в каждом. 

# Не нужно знать высшую математику, чтобы понять, что в таком случае мы увеличим размерность исходной 
# таблицы ещё на сотню и 99 % строк этих столбцов будут заполнены нулями (False). При увеличении размерности 
# в данных время обучения некоторых моделей растет экспоненциально — увеличив число признаков в два раза, вы увеличите время обучения 
# в 7.38 раза! И при этом, возможно, даже не получите прироста в качестве. 

# Более того, существует такое понятие, как проклятие размерности. Когда мы с вами будем обучать модели, 
# мы будем искать глобальный минимум некоторой функции потерь. Размерность этой функции определяется числом признаков, 
# на которых мы обучаем алгоритм. 

# Проклятие размерности гласит, что, увеличивая размерность функции, мы повышаем сложность поиска этого минимума 
# и рискуем вовсе не найти его! Об этом страшном проклятии мы ещё будем говорить в курсе по ML и даже попробуем его победить. 
# Но об этом чуть позже.

# Однако, несмотря на свои недостатки, этот метод кажется наиболее логичным из предложенных ранее и часто используется в очистке данных.

# КОМБИНИРОВАНИЕ МЕТОДОВ

# Наверняка вы уже догадались, что необязательно использовать один метод. Вы можете их комбинировать. Например, мы можем:

# удалить столбцы, в которых более 30 % пропусков;
# удалить записи, в которых более двух пропусков одновременно;
# заполнить оставшиеся ячейки константами.
# Посмотрим на реализацию такого подхода в коде:

#создаём копию исходной таблицы
combine_data = sber_data.copy()

#отбрасываем столбцы с числом пропусков более 30% (100-70)
n = combine_data.shape[0] #число строк в таблице
thresh = n*0.7
combine_data = combine_data.dropna(how='any', thresh=thresh, axis=1)

#отбрасываем строки с числом пропусков более 2 в строке
m = combine_data.shape[1] #число признаков после удаления столбцов
combine_data = combine_data.dropna(how='any', thresh=m-2, axis=0)

#создаём словарь 'имя_столбца': число (признак), на который надо заменить пропуски 
values = {
    'life_sq': combine_data['full_sq'],
    'metro_min_walk': combine_data['metro_min_walk'].median(),
    'metro_km_walk': combine_data['metro_km_walk'].median(),
    'railroad_station_walk_km': combine_data['railroad_station_walk_km'].median(),
    'railroad_station_walk_min': combine_data['railroad_station_walk_min'].median(),
    'preschool_quota': combine_data['preschool_quota'].mode()[0],
    'school_quota': combine_data['school_quota'].mode()[0],
    'floor': combine_data['floor'].mode()[0]
}
#заполняем оставшиеся записи константами в соответствии со словарем values
combine_data = combine_data.fillna(values)
#выводим результирующую долю пропусков
print(combine_data.isnull().mean())

# Выведем результирующее число строк и столбцов:
print(combine_data.shape)

In [ ]:
# ЗАДАНИЕ 4.6

df = sber_data.copy()
thresh = df.shape[0]*0.5
df = df.dropna(thresh=thresh, axis=1)
thresh2 = df.shape[1] - 2
df = df.dropna(thresh=thresh2, axis=0)
df = df.fillna({
    'metro_min_walk': df['metro_min_walk'].mean(),
    'railroad_station_walk_min': df['railroad_station_walk_min'].mean(),
    'school_quota': df['school_quota'].mode()[0]})
print(df.isnull().mean())

### 5. Выбросы: почему появляются и чем опасны?

In [ ]:
# МЕТОД РУЧНОГО ПОИСКА И ЗДРАВОГО СМЫСЛА

# Это самый трудоёмкий метод, основанный на житейской логике, методе пристального взгляда и небольшом количестве статистики. 
# Он предполагает поиск невозможных и нелогичных значений в данных.

# # Пусть у нас есть признак, по которому мы будем искать выбросы. 
# Давайте рассчитаем его статистические показатели (минимум, максимум, среднее, квантили) и по ним попробуем определить наличие аномалий.

# Сделать это можно с помощью уже знакомого вам метода describe(). 
# Рассчитаем статистические показатели для признака жилой площади (life_sq).
sber_data['life_sq'].describe()

In [ ]:
# Что нам говорит метод describe()? Во-первых, у нас есть квартиры с нулевой жилой площадью. 
# Во-вторых, в то время как 75-й квантиль равен 43, максимум превышает 7 тысяч квадратных метров (целый дворец, а не квартира!). 
# Найдём число квартир с нулевой жилой площадью:
print(sber_data[sber_data['life_sq'] == 0].shape[0])

In [ ]:
# А теперь выведем здания с жилой площадью более 7 000 квадратных метров:
print(sber_data[sber_data['life_sq'] > 7000])

In [ ]:
# Вот он, красавец! Выброс налицо: гигантская жилая площадь (life_sq), да ещё почти в 100 раз превышает общую площадь (full_sq).
# Логичен вопрос: а много ли у нас таких квартир, у которых жилая площадь больше, чем суммарная?
# Давайте проверим это с помощью фильтрации:
outliers = sber_data[sber_data['life_sq'] > sber_data['full_sq']]
print(outliers.shape[0])

In [ ]:
# Таких квартир оказывается 37 штук. Подобные наблюдения уже не поддаются здравому смыслу — они являются ошибочными, 
# и от них стоит избавиться. Для этого можно воспользоваться методом drop() и удалить записи по их индексам:
cleaned = sber_data.drop(outliers.index, axis=0)
print(f'Результирующее число записей: {cleaned.shape[0]}')

In [ ]:
# Ещё пример: давайте посмотрим на признак числа этажей (floor).
print(sber_data['floor'].describe())

# Снова видим подозрительную максимальную отметку в 77 этажей. Проверим все квартиры, которые находятся выше 50 этажей:
print(sber_data[sber_data['floor']> 50])

In [ ]:
# МЕТОД МЕЖКВАРТИЛЬНОГО РАЗМАХА (МЕТОД ТЬЮКИ)

# Отличным помощником в поиске потенциальных выбросов является визуализация. 
# Если признак является числовым, то можно построить гистограмму или коробчатую диаграмму, чтобы найти аномалии.

# На гистограмме мы можем увидеть потенциальные выбросы как низкие далеко отстоящие от основной группы столбцов «пеньки», 
# а на коробчатой диаграмме — точки за пределами усов.

# Построим гистограмму и коробчатую диаграмму для признака полной площади (full_sq):
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(15, 4))
histplot = sns.histplot(data=sber_data, x='full_sq', ax=axes[0]);
histplot.set_title('Full Square Distribution');
boxplot = sns.boxplot(data=sber_data, x='full_sq', ax=axes[1]);
boxplot.set_title('Full Square Boxplot');

In [ ]:
# Алгоритм метода:

# вычислить 25-ый и 75-ый квантили (первый и третий квартили) — и  для признака, который мы исследуем;
# вычислить межквартильное расстояние: ;
# вычислить верхнюю и нижнюю границы Тьюки: 

# найти наблюдения, которые выходят за пределы границ.

# В соответствии с этим алгоритмом напишем функцию outliers_iqr(), которая вам может ещё не раз пригодиться в реальных задачах. 
# Эта функция принимает на вход DataFrame и признак, по которому ищутся выбросы, а затем возвращает потенциальные выбросы, 
# найденные с помощью метода Тьюки, и очищенный от них датасет.

# Квантили вычисляются с помощью метода quantile(). 
# Потенциальные выбросы определяются при помощи фильтрации данных по условию выхода за пределы верхней или нижней границы.
def outliers_iqr(data, feature):
    x = data[feature]
    quartile_1, quartile_3 = x.quantile(0.25), x.quantile(0.75),
    iqr = quartile_3 - quartile_1
    lower_bound = quartile_1 - (iqr * 1.5)
    upper_bound = quartile_3 + (iqr * 1.5)
    outliers = data[(x<lower_bound) | (x > upper_bound)]
    cleaned = data[(x>lower_bound) & (x < upper_bound)]
    return outliers, cleaned

# Применим эту функцию к таблице sber_data и признаку full_sq, а также выведем размерности результатов:
outliers, cleaned = outliers_iqr(sber_data, 'full_sq')
print(f'Число выбросов по методу Тьюки: {outliers.shape[0]}')
print(f'Результирующее число записей: {cleaned.shape[0]}')

# Согласно классическому методу Тьюки, под выбросы у нас попали 963 записи в таблице. 
# Давайте построим гистограмму и коробчатую диаграмму на новых данных cleaned_sber_data:
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(15, 4))
histplot = sns.histplot(data=cleaned, x='full_sq', ax=axes[0]);
histplot.set_title('Cleaned Full Square Distribution');
boxplot = sns.boxplot(data=cleaned, x='full_sq', ax=axes[1]);
boxplot.set_title('Cleaned Full Square Boxplot');

In [ ]:
# ЗАДАНИЕ 6.1
# Давайте немного модифицируем функцию outliers_iqr(). Добавьте в неё параметры left и right, 
# которые задают число IQR влево и вправо от границ ящика (пусть по умолчанию они равны 1.5). 
# Функция, как и раньше, должна возвращать потенциальные выбросы и очищенный DataFrame.
def outliers_iqr_mod(data, feature, left=1.5, right=1.5):
    x = data[feature]
    quartile_1, quartile_3 = x.quantile(0.25), x.quantile(0.75),
    iqr = quartile_3 - quartile_1
    lower_bound = quartile_1 - (iqr * left)
    upper_bound = quartile_3 + (iqr * right)
    outliers = data[(x<lower_bound) | (x> upper_bound)]
    cleaned = data[(x>lower_bound) & (x < upper_bound)]
    return outliers, cleaned

sber_data = pd.read_csv('data/sber_data.csv')

In [ ]:
# МЕТОД Z-ОТКЛОНЕНИЙ (МЕТОД СИГМ)

# Последний метод, который мы рассмотрим, — это метод, основанный на правиле трёх сигм для нормального распределения. 

# Правило трёх сигм гласит: если распределение данных является нормальным, 
# то 99,73 % лежат в интервале от , где   (мю) — математическое ожидание (для выборки это среднее значение), 
# а (сигма) — стандартное отклонение. Наблюдения, которые лежат за пределами этого интервала, будут считаться выбросами.

# А что делать, если данные не распределены нормально? 

# На такой случай есть один трюк. Иногда для распределений, похожих на логнормальное, может помочь логарифмирование.
# Оно может привести исходное распределение к подобию нормального. Причем, основание логарифма может быть любым.

# Рассмотрим логарифмирование на примере. 
# Построим две гистограммы признака расстояния до МКАД (mkad_km): первая — в обычном масштабе, 
# а вторая — в логарифмическом. Логарифмировать будем с помощью функции log() из библиотеки numpy 
# (натуральный логарифм — логарифм по основанию числа e). Признак имеет среди своих значений 0. 
# Из математики известно, что логарифма от 0 не существует, поэтому мы прибавляем к нашему признаку 1,
# чтобы не логарифмировать нули и не получать предупреждения.
fig, axes = plt.subplots(1, 2, figsize=(15, 4))

#гистограмма исходного признака
histplot = sns.histplot(sber_data['mkad_km'], bins=30, ax=axes[0])
histplot.set_title('MKAD Km Distribution');

#гистограмма в логарифмическом масштабе
log_mkad_km= np.log(sber_data['mkad_km'] + 1)
histplot = sns.histplot(log_mkad_km , bins=30, ax=axes[1])
histplot.set_title('Log MKAD Km Distribution');

In [ ]:
# Давайте реализуем алгоритм метода z-отклонения. Описание алгоритма метода:

# вычислить математическое ожидание  (среднее) и стандартное отклонение  признака ;
# вычислить нижнюю и верхнюю границу интервала как:
# найти наблюдения, которые выходят за пределы границ.

# Напишем функцию outliers_z_score(), которая реализует этот алгоритм. 

# На вход она принимает DataFrame и признак, по которому ищутся выбросы. 
# В дополнение добавим в функцию возможность работы в логарифмическом масштабе: для этого введём аргумент log_scale. 
# Если он равен True, то будем логарифмировать рассматриваемый признак, иначе — оставляем его в исходном виде.

# Как и раньше, функция будет возвращать выбросы и очищенные от них данные:
def outliers_z_score(data, feature, log_scale=False):
    if log_scale:
        x = np.log(data[feature]+1)
    else:
        x = data[feature]
    mu = x.mean()
    sigma = x.std()
    lower_bound = mu - 3 * sigma
    upper_bound = mu + 3 * sigma
    outliers = data[(x < lower_bound) | (x > upper_bound)]
    cleaned = data[(x > lower_bound) & (x < upper_bound)]
    return outliers, cleaned

# Применим эту функцию к таблице sber_data и признаку mkad_km, а также выведем размерности результатов:
outliers, cleaned = outliers_z_score(sber_data, 'mkad_km', log_scale=True)
print(f'Число выбросов по методу z-отклонения: {outliers.shape[0]}')
print(f'Результирующее число записей: {cleaned.shape[0]}')

# Итак, метод z-отклонения нашел нам 33 потенциальных выброса по признаку расстояния до МКАД. 
# Давайте узнаем, в каких районах (sub_area) представлены эти квартиры:
print(outliers['sub_area'].unique())

In [ ]:
# Наши потенциальные выбросы — это квартиры из поселений «Роговское» и «Киевский». 
# Снова обращаемся к силе интернета и «пробиваем» наших подозреваемых. 
# Эти поселения — самые удалённые районы Московской области; первое из них — это и вовсе граница с Калужской областью. 

# И тут возникает закономерный вопрос: а стоит ли считать такие наблюдения за выбросы? 
# Вопрос в действительности не имеет определенного ответа: с одной стороны, метод прямо-таки говорит нам об этом, 
# а с другой — эти наблюдения имеют право на существование, ведь они являются частью Московской области.

# Возможно, мы не учли того факта, что наш логарифм распределения всё-таки не идеально нормален 
# и в нём присутствует некоторая асимметрия. Возможно, стоит дать некоторое «послабление» на границы интервалов? 
# Давайте отдельно построим гистограмму прологарифмированного распределения, а также отобразим на гистограмме вертикальные линии, 
# соответствующие среднему (центру интервала в методе трёх сигм) и границы интервала . 
# Вертикальные линии можно построить с помощью метода axvline(). Для среднего линия будет обычной, 
# а для границ интервала — пунктирной (параметр ls ='--'):
fig, ax = plt.subplots(1, 1, figsize=(8, 4))
log_mkad_km = np.log(sber_data['mkad_km'] + 1)
histplot = sns.histplot(log_mkad_km, bins=30, ax=ax)
histplot.axvline(log_mkad_km.mean(), color='k', lw=2)
histplot.axvline(log_mkad_km.mean()+ 3 * log_mkad_km.std(), color='k', ls='--', lw=2)
histplot.axvline(log_mkad_km.mean()- 3 * log_mkad_km.std(), color='k', ls='--', lw=2)
histplot.set_title('Log MKAD Km Distribution');

# Итак, что мы графически построили интервал метода трёх сигм поверх нашего распределения. 
# Он показывает, какие наблюдения мы берем в интервал, а какие считаем выбросами. 
# Легко заметить, среднее значение (жирная вертикальная линия) находится левее моды, 
# это свойство распределений с левосторонней асимметрией. Также видны наблюдения, 
# которые мы не захватили своим интервалом (небольшой пенек правее верхней границы) — это и есть наши квартиры из из поселений 
# "Роговское" и "Киевский". Очевидно, что если немного (меньше чем на одну сигму) "сдвинуть" верхнюю границу вправо, 
# мы захватим эти наблюдения. Давайте сделаем это?!

In [ ]:
# ЗАДАНИЕ 6.5
# Постройте гистограмму для признака price_doc в логарифмическом масштабе. 
# А также, добавьте на график линии, отображающие среднее и границы интервала для метода трех сигм. Выберите верные утверждения:
fig, ax = plt.subplots(1, 1, figsize=(8, 4))
log_price = np.log(sber_data['price_doc'])
histplot = sns.histplot(log_price, bins=30, ax=ax)
histplot.set_title('Log Price Distribution');
histplot.axvline(log_price.mean(), color='k', lw=2)
histplot.axvline(log_price.mean()+ 3 * log_price.std(), color='k', ls='--', lw=2)
histplot.axvline(log_price.mean()- 3 * log_price.std(), color='k', ls='--', lw=2);

### 7. Работа с дубликатами и неинформативными признаками

In [ ]:
# ОБНАРУЖЕНИЕ И ЛИКВИДАЦИЯ ДУБЛИКАТОВ

# Способ обнаружения дубликатов зависит от того, что именно вы считаете дубликатом. 
# Например, за дубликаты можно посчитать записи, у которых совпадают все признаки или их часть.
# Если в таблице есть столбец с уникальным идентификатором (id), вы можете попробовать поискать дубликаты по нему: 
# одинаковые записи могут иметь одинаковый id.

# Проверим, есть у нас такие записи: для этого сравним число уникальных значений в столбце id с числом строк. 
# Число уникальных значений вычислим с помощью метода nunique():
sber_data['id'].nunique() == sber_data.shape[0]

In [ ]:
# Вроде бы всё в порядке: каждой записи в таблице соответствует свой уникальный идентификатор. 
# Но это ещё не означает, что в таблице нет дубликатов!

# Столбец с id задаёт каждой строке свой уникальный номер, поэтому сама по себе каждая строка является уникальной. 
# Однако содержимое других столбцов может повторяться.

# Чтобы отследить дубликаты, можно воспользоваться методом duplicated(), который возвращает булеву маску для фильтрации. 
# Для записей, у которых совпадают признаки, переданные методу, он возвращает True, для остальных — False.

# У метода есть параметр subset — список признаков, по которым производится поиск дубликатов. 
# По умолчанию используются все столбцы в DataFrame и ищутся полные дубликаты.

# Найдём число полных дубликатов таблице sber_data. Предварительно создадим список столбцов dupl_columns, 
# по которым будем искать совпадения (все столбцы, не включая id). 

# Создадим маску дубликатов с помощью метода duplicated() и произведём фильтрацию. 
# Результат заносим в переменную sber_duplicates. Выведем число строк в результирующем DataFrame:
dupl_columns = list(sber_data.columns)
dupl_columns.remove('id')

mask = sber_data.duplicated(subset=dupl_columns)
sber_duplicates = sber_data[mask]
print(f'Число найденных дубликатов: {sber_duplicates.shape[0]}')

In [41]:
# Итак, 562 строки в таблице являются полными копиями других записей. 
# Ручной поиск совпадающих строк по 30 тысячам записей был бы практически невыполним, 
# а с помощью pandas мы быстро, а главное, легко обнаружили дублирующиеся данные!

# Теперь нам необходимо от них избавиться. Для этого легче всего воспользоваться методом drop_duplicates(),
# который удаляет повторяющиеся записи из таблицы. 

# Создадим новую таблицу sber_dedupped, которая будет версией исходной таблицы, очищенной от полных дубликатов.
sber_dedupped = sber_data.drop_duplicates(subset=dupl_columns)
print(f'Результирующее число записей: {sber_dedupped.shape[0]}')

Результирующее число записей: 29909


In [42]:
# НЕИНФОРМАТИВНЫЕ ПРИЗНАКИ

# Отсутствием новой и полезной информации могут «похвастаться» не только отдельные записи в таблице, но и целые признаки.

# Неинформативными называются признаки, в которых большая часть строк содержит одинаковые значения 
# (например, пол клиентов в мужском барбершопе), либо наоборот — признак, в котором для большинства записей значения уникальны
# (например, номер телефона клиента). 

# Мы уже сталкивались с неинформативными признаками. Вспомните датасет о домах в Мельбурне, 
# где был признак адреса дома с уникальным значением для каждого дома. 

# ОБНАРУЖЕНИЕ И ЛИКВИДАЦИЯ НЕИНФОРМАТИВНЫХ ПРИЗНАКОВ

# Чтобы считать признак неинформативным, прежде всего нужно задать какой-то определённый порог. 
# Например, часто используют пороги в 0.95 и 0.99. Это означает: признак неинформативен, 
# если в нем 95 % (99 %) одинаковых значений или же 95 % (99 %) данных полностью уникальны.

# К сожалению, в pandas пока нет волшебной палочки, которая мгновенно бы выдавала список столбцов, 
# обладающих низкой информативностью. Однако процедура их поиска легко реализуется вручную.

# Разберём алгоритм:
# Создаём пустой список low_information_cols, куда будем добавлять названия признаков, которые мы посчитаем неинформативными.

# В цикле пройдёмся по всем именам столбцов в таблице и для каждого будем совершать следующие действия:
# рассчитаем top_freq — наибольшую относительную частоту с помощью метода value_counts() 
# с параметром normalize=True. Метод вернёт долю от общих данных, которую занимает каждое уникальное значение в признаке.

#список неинформативных признаков
low_information_cols = [] 

#цикл по всем столбцам
for col in sber_data.columns:
    #наибольшая относительная частота в признаке
    top_freq = sber_data[col].value_counts(normalize=True).max()
    #доля уникальных значений от размера признака
    nunique_ratio = sber_data[col].nunique() / sber_data[col].count()
    # сравниваем наибольшую частоту с порогом
    if top_freq > 0.95:
        low_information_cols.append(col)
        print(f'{col}: {round(top_freq*100, 2)}% одинаковых значений')
    # сравниваем долю уникальных значений с порогом
    if nunique_ratio > 0.95:
        low_information_cols.append(col)
        print(f'{col}: {round(nunique_ratio*100, 2)}% уникальных значений')

id: 100.0% уникальных значений
oil_chemistry_raion: 99.03% одинаковых значений
railroad_terminal_raion: 96.27% одинаковых значений
nuclear_reactor_raion: 97.17% одинаковых значений
big_road1_1line: 97.44% одинаковых значений
mosque_count_1000: 98.08% одинаковых значений


In [43]:
# Итак, мы нашли шесть неинформативных признаков. Теперь можно удалить их с помощью метода drop(),
# передав результирующий список в его аргументы.
information_sber_data = sber_data.drop(low_information_cols, axis=1)
print(f'Результирующее число признаков: {information_sber_data.shape[1]}')

Результирующее число признаков: 55
